In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import time 

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['champs-scalar-coupling', 'predmolprop-featureengineering-slow']


In [2]:
# import data
#train = pd.read_csv("../input/champs-scalar-coupling/train.csv")
#test = pd.read_csv("../input/champs-scalar-coupling/test.csv")
structures = pd.read_csv("../input/champs-scalar-coupling/structures.csv")
train_extend = pd.read_csv("../input/predmolprop-featureengineering-slow/train_extend.csv")
test_extend = pd.read_csv("../input/predmolprop-featureengineering-slow/test_extend.csv")

In [3]:
train_extend.columns

Index(['id', 'molecule_name', 'atom_index_0', 'atom_index_1', 'type',
       'scalar_coupling_constant', 'num_bonds', 'atom_end_type',
       'num_mol_bonds', 'min_d', 'mean_d', 'max_d', 'space_dr', 'bond_dr',
       'bond_1', 'bond_2', 'bond_3', 'atom_0_pc', 'atom_end_pc', 'atom_0_fc',
       'atom_end_fc', 'atom_0_val', 'atom_end_val', 'atom_0_sm', 'atom_end_sm',
       'atom_0_type2', 'atom_2_type', 'atom_3_type', 'atom_end_type2',
       'atom_2_hyb', 'atom_3_hyb', 'atom_end_hyb', 'path_count', 'atom_0_min',
       'atom_0_mean', 'atom_0_max', 'atom_0_Cmin', 'atom_0_Cmean',
       'atom_0_Cmax', 'atom_0_Omin', 'atom_0_Omean', 'atom_0_Omax',
       'atom_0_Nmin', 'atom_0_Nmean', 'atom_0_Nmax', 'atom_0_Fmin',
       'atom_0_Fmean', 'atom_0_Fmax', 'atom_end_min', 'atom_end_mean',
       'atom_end_max', 'atom_end_Cmin', 'atom_end_Cmean', 'atom_end_Cmax',
       'atom_end_Omin', 'atom_end_Omean', 'atom_end_Omax', 'atom_end_Nmin',
       'atom_end_Nmean', 'atom_end_Nmax', 'atom_end_Fmin'

In [4]:
# there are 3 bond couplings with no defined dihedral angle because they are in linear molecules
# define the dihedral angle as -1 and make indicator column
def FindNan(x):
    if str(x)=='nan':
        return 1
    else:
        return 0

def ReplaceNan(x):
    if str(x)=='nan':
        return -2
    else:
        return x

train_extend['is_linear']=train_extend.bond3_angle.map(FindNan)
train_extend['bond3_angle']=train_extend.bond3_angle.map(ReplaceNan)

test_extend['is_linear']=test_extend.bond3_angle.map(FindNan)
test_extend['bond3_angle']=test_extend.bond3_angle.map(ReplaceNan)

Count the atoms of each type

In [5]:
def CleanColsAndIdx(df):
    df.columns = df.columns.set_names(None)
    df.index = df.index.set_names(None)
    return df

In [6]:
# Group by molecule_name and atom, then use size() to count how many of each atom
# Then use unstack to make the atom types, which groupby made into indices, into columns
# Finally, because not all molecule contain all five atoms, fill NAs with 0
AtomTypes=structures.groupby(['molecule_name','atom'],sort=False).size().unstack('atom').fillna(0)
# remove index names (optional)

# rename columns
AtomTypes = AtomTypes.rename(columns={'C':'num_C','H':'num_H','N':'num_N',
                                                  'O':'num_O','F':'num_F'})

AtomTypes=CleanColsAndIdx(AtomTypes)

AtomTypes=AtomTypes.astype('int8')

Measurements from the Center of Mass (COM)

In [7]:
gb=structures.groupby('molecule_name',sort=False)
NumAtoms=pd.DataFrame(gb.size(),columns=['total_atoms'])

COM=gb.mean()
COM.pop('atom_index')
COM = COM.rename(columns={'x':'COM_x','y':'COM_y','z':'COM_z'})
COM = CleanColsAndIdx(COM)
structures = pd.merge(structures,COM,how='left',left_on='molecule_name',right_index=True)

axis = ['x','y','z']
for ax in axis:
    structures['d'+ax]=structures[ax]-structures['COM_'+ax]

structures=structures.assign(COM_dr=lambda x: np.sqrt(x.dx**2+x.dy**2+x.dz**2))

gb=structures[['molecule_name','COM_dr']].groupby('molecule_name',sort=False)
Dmin_COM=gb.min(); Dmin_COM.columns=['Dmin_COM']
Dmin_COM=CleanColsAndIdx(Dmin_COM)
Dmean_COM=gb.mean(); Dmean_COM.columns=['Dmean_COM']
Dmean_COM=CleanColsAndIdx(Dmean_COM)
Dmax_COM=gb.max(); Dmax_COM.columns=['Dmax_COM']
Dmax_COM=CleanColsAndIdx(Dmax_COM)

In [8]:
poplist = ['COM_x','COM_y','COM_z','dx','dy','dz']#,'x','y','z']
for col in poplist:
    structures.pop(col)

structures.head(20)

,molecule_name,atom_index,atom,x,y,z,COM_dr
0,dsgdb9nsd_000001,0,C,-0.012698,1.085804,0.008001,0.000012
1,dsgdb9nsd_000001,1,H,0.002150,-0.006031,0.001976,1.091945
2,dsgdb9nsd_000001,2,H,1.011731,1.463751,0.000277,1.091946
3,dsgdb9nsd_000001,3,H,-0.540815,1.447527,-0.876644,1.091954
4,dsgdb9nsd_000001,4,H,-0.523814,1.437933,0.906397,1.091954
5,dsgdb9nsd_000002,0,N,-0.040426,1.024108,0.062564,0.301261
6,dsgdb9nsd_000002,1,H,0.017257,0.012545,-0.027377,0.939879
7,dsgdb9nsd_000002,2,H,0.915789,1.358745,-0.028758,0.939876
8,dsgdb9nsd_000002,3,H,-0.520278,1.343532,-0.775543,0.939962
9,dsgdb9nsd_000003,0,O,-0.034360,0.977540,0.007602,0.396141


In [9]:
MolProps = AtomTypes.copy()
MolProps=MolProps.join([NumAtoms,Dmin_COM,Dmean_COM,Dmax_COM])
del AtomTypes,NumAtoms,COM,Dmin_COM,Dmean_COM,Dmax_COM
MolProps.head(20)

,num_C,num_H,num_N,num_O,num_F,total_atoms,Dmin_COM,Dmean_COM,Dmax_COM
dsgdb9nsd_000001,1,4,0,0,0,5,0.000012,0.873562,1.091954
dsgdb9nsd_000002,0,3,1,0,0,4,0.301261,0.780244,0.939962
dsgdb9nsd_000003,0,2,0,1,0,3,0.396141,0.653496,0.782173
dsgdb9nsd_000004,2,2,0,0,0,4,0.599539,1.130589,1.661639
dsgdb9nsd_000005,1,1,1,0,0,3,0.028383,0.748910,1.123365
dsgdb9nsd_000007,2,6,0,0,0,8,0.764813,1.352176,1.547991
dsgdb9nsd_000008,1,4,0,1,0,6,0.354950,1.136816,1.586039
dsgdb9nsd_000009,3,4,0,0,0,7,0.506195,1.571041,2.769087
dsgdb9nsd_000010,2,3,1,0,0,6,0.488208,1.268453,2.123730
dsgdb9nsd_000011,2,4,0,1,0,7,0.650007,1.337038,1.841725


In [10]:
def map_atom_info(df, atom_idx):
    df = pd.merge(df, structures, how = 'left',
                  left_on  = ['molecule_name', f'atom_index_{atom_idx}'],
                  right_on = ['molecule_name',  'atom_index'])
    
    df = df.drop(['atom_index','atom','x','y','z'], axis=1)
    df = df.rename(columns={'COM_dr': f'COM_dr_{atom_idx}'})
    return df

train_extend = map_atom_info(train_extend, 0)
train_extend = map_atom_info(train_extend, 1)

test_extend = map_atom_info(test_extend, 0)
test_extend = map_atom_info(test_extend, 1)

In [11]:
def map_mol_info(df):
    df = pd.merge(df, MolProps, how = 'left',
                  left_on  = ['molecule_name'],
                  right_index = True)
    return df

train_extend = map_mol_info(train_extend)
test_extend = map_mol_info(test_extend)

In [12]:
train_extend.columns

Index(['id', 'molecule_name', 'atom_index_0', 'atom_index_1', 'type',
       'scalar_coupling_constant', 'num_bonds', 'atom_end_type',
       'num_mol_bonds', 'min_d', 'mean_d', 'max_d', 'space_dr', 'bond_dr',
       'bond_1', 'bond_2', 'bond_3', 'atom_0_pc', 'atom_end_pc', 'atom_0_fc',
       'atom_end_fc', 'atom_0_val', 'atom_end_val', 'atom_0_sm', 'atom_end_sm',
       'atom_0_type2', 'atom_2_type', 'atom_3_type', 'atom_end_type2',
       'atom_2_hyb', 'atom_3_hyb', 'atom_end_hyb', 'path_count', 'atom_0_min',
       'atom_0_mean', 'atom_0_max', 'atom_0_Cmin', 'atom_0_Cmean',
       'atom_0_Cmax', 'atom_0_Omin', 'atom_0_Omean', 'atom_0_Omax',
       'atom_0_Nmin', 'atom_0_Nmean', 'atom_0_Nmax', 'atom_0_Fmin',
       'atom_0_Fmean', 'atom_0_Fmax', 'atom_end_min', 'atom_end_mean',
       'atom_end_max', 'atom_end_Cmin', 'atom_end_Cmean', 'atom_end_Cmax',
       'atom_end_Omin', 'atom_end_Omean', 'atom_end_Omax', 'atom_end_Nmin',
       'atom_end_Nmean', 'atom_end_Nmax', 'atom_end_Fmin'

In [13]:
len(train_extend.columns)

77

In [14]:
test_extend.atom_2_type.unique()

array(['C1', nan, 'C3', 'Nam', 'O3', 'C2', 'N3', 'Car', 'O2', 'Nar',
       'Npl', 'N2', 'Ng+', 'N3+'], dtype=object)

In [15]:
train_extend.atom_3_type.unique()

array([nan, 'C3', 'O3', 'C1', 'C2', 'N2', 'Car', 'Nar', 'O2', 'N3', 'Nam',
       'Npl', 'C+', 'N3+', 'Nox', 'Ng+'], dtype=object)

In [16]:
# check that the categorical features are the same
print(len(set(train_extend.atom_2_type.unique())-set(test_extend.atom_2_type.unique())))
print(len(set(test_extend.atom_3_type.unique())-set(train_extend.atom_3_type.unique())))

0
0


In [17]:
train_extend[train_extend.columns[6:26]].describe()

,num_bonds,num_mol_bonds,min_d,mean_d,max_d,space_dr,bond_dr,bond_1,bond_2,bond_3,atom_0_pc,atom_end_pc,atom_0_fc,atom_end_fc,atom_0_val,atom_end_val,atom_0_sm,atom_end_sm
count,4.658147e+06,4.658147e+06,4.658147e+06,4.658147e+06,4.658147e+06,4.658147e+06,4.658147e+06,4658147.0,4.658147e+06,4.658147e+06,4.658147e+06,4.658147e+06,4658147.0,4658147.0,4658147.0,4.658147e+06,4658147.0,4.658147e+06
mean,2.325156e+00,1.975888e+01,1.029168e+00,3.255689e+00,6.589293e+00,2.360922e+00,2.987136e+00,1.0,7.078817e-01,6.478327e-03,5.058974e-02,1.225711e-02,0.0,0.0,1.0,3.121726e+00,0.0,1.211061e-02
std,7.366406e-01,2.875716e+00,5.350769e-02,2.638297e-01,9.869911e-01,7.367096e-01,1.083377e+00,0.0,7.746804e-01,1.053283e+00,4.177597e-02,1.057742e-01,0.0,0.0,0.0,1.202994e+00,0.0,1.551868e-01
min,1.000000e+00,2.000000e+00,9.586066e-01,1.145857e+00,1.513358e+00,1.002241e+00,1.002241e+00,1.0,-1.000000e+00,-1.000000e+00,-9.007463e-02,-4.188789e-01,0.0,0.0,1.0,1.000000e+00,0.0,0.000000e+00
25%,2.000000e+00,1.800000e+01,9.646925e-01,3.080365e+00,5.868474e+00,1.948526e+00,2.363860e+00,1.0,1.000000e+00,-1.000000e+00,2.721544e-02,-3.294718e-02,0.0,0.0,1.0,2.000000e+00,0.0,0.000000e+00
50%,2.000000e+00,2.000000e+01,1.018438e+00,3.222618e+00,6.450707e+00,2.313485e+00,2.657140e+00,1.0,1.000000e+00,-1.000000e+00,3.420534e-02,2.626475e-02,0.0,0.0,1.0,4.000000e+00,0.0,0.000000e+00
75%,3.000000e+00,2.200000e+01,1.084557e+00,3.390371e+00,7.113688e+00,2.946476e+00,4.041835e+00,1.0,1.000000e+00,1.000000e+00,5.753240e-02,6.224376e-02,0.0,0.0,1.0,4.000000e+00,0.0,0.000000e+00
max,3.000000e+00,2.800000e+01,1.111452e+00,5.345653e+00,1.204039e+01,3.924354e+00,4.448899e+00,1.0,3.000000e+00,3.000000e+00,3.640839e-01,5.500642e-01,0.0,0.0,1.0,4.000000e+00,0.0,3.000000e+00


In [18]:
train_extend[train_extend.columns[26:46]].describe()

,atom_2_hyb,atom_3_hyb,atom_end_hyb,path_count,atom_0_min,atom_0_mean,atom_0_max,atom_0_Cmin,atom_0_Cmean,atom_0_Cmax,atom_0_Omin,atom_0_Omean,atom_0_Omax,atom_0_Nmin,atom_0_Nmean,atom_0_Nmax,atom_0_Fmin
count,4.658147e+06,4.658147e+06,4.658147e+06,4.658147e+06,4.658147e+06,4.658147e+06,4.658147e+06,4.658147e+06,4.658147e+06,4.658147e+06,4.658147e+06,4.658147e+06,4.658147e+06,4.658147e+06,4.658147e+06,4.658147e+06,4.658147e+06
mean,2.271297e+00,8.566037e-01,2.174139e+00,1.034380e+00,1.086151e+00,3.448149e+00,5.583775e+00,1.156816e+00,3.000913e+00,4.603247e+00,2.495728e+00,2.802087e+00,3.110704e+00,1.207891e+00,1.398023e+00,1.582726e+00,-9.686235e-01
std,1.464550e+00,1.929500e+00,1.201639e+00,1.895493e-01,2.592215e-02,4.241759e-01,9.728497e-01,2.367558e-01,4.678379e-01,9.935855e-01,1.947983e+00,2.017329e+00,2.213152e+00,2.218438e+00,2.346820e+00,2.548368e+00,4.059599e-01
min,-1.000000e+00,-1.000000e+00,0.000000e+00,1.000000e+00,9.586066e-01,1.237732e+00,1.513358e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00
25%,3.000000e+00,-1.000000e+00,2.000000e+00,1.000000e+00,1.087620e+00,3.153603e+00,4.921539e+00,1.090556e+00,2.674816e+00,3.956129e+00,2.055108e+00,2.333356e+00,2.457948e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00
50%,3.000000e+00,-1.000000e+00,3.000000e+00,1.000000e+00,1.093561e+00,3.370020e+00,5.422875e+00,1.094228e+00,2.913418e+00,4.454278e+00,2.718270e+00,3.267650e+00,3.601330e+00,1.021719e+00,2.114150e+00,2.132134e+00,-1.000000e+00
75%,3.000000e+00,3.000000e+00,3.000000e+00,1.000000e+00,1.095879e+00,3.651209e+00,6.146281e+00,1.096858e+00,3.232154e+00,5.184237e+00,3.801027e+00,4.079756e+00,4.540916e+00,3.039998e+00,3.403922e+00,3.815579e+00,-1.000000e+00
max,3.000000e+00,3.000000e+00,3.000000e+00,3.000000e+00,1.247942e+00,7.585518e+00,1.204039e+01,3.496963e+00,6.523529e+00,1.115916e+01,1.092534e+01,1.092534e+01,1.092534e+01,1.075381e+01,1.075381e+01,1.075381e+01,7.805846e+00


In [19]:
train_extend[train_extend.columns[46:]].describe()

,atom_0_Fmean,atom_0_Fmax,atom_end_min,atom_end_mean,atom_end_max,atom_end_Cmin,atom_end_Cmean,atom_end_Cmax,atom_end_Omin,atom_end_Omean,...,COM_dr_1,num_C,num_H,num_N,num_O,num_F,total_atoms,Dmin_COM,Dmean_COM,Dmax_COM
count,4.658147e+06,4.658147e+06,4.658147e+06,4.658147e+06,4.658147e+06,4.658147e+06,4.658147e+06,4.658147e+06,4.658147e+06,4.658147e+06,...,4.658147e+06,4.658147e+06,4.658147e+06,4.658147e+06,4.658147e+06,4.658147e+06,4.658147e+06,4.658147e+06,4.658147e+06,4.658147e+06
mean,-9.673718e-01,-9.660134e-01,1.142591e+00,2.954799e+00,4.936891e+00,1.432461e+00,2.623027e+00,3.946114e+00,2.049699e+00,2.336697e+00,...,1.805007e+00,6.719829e+00,1.021538e+01,8.213094e-01,1.276229e+00,1.056364e-02,1.904331e+01,7.722112e-01,2.290601e+00,3.620758e+00
std,4.197352e-01,4.363600e-01,1.209999e-01,4.779996e-01,1.056066e+00,2.299194e-01,5.494215e-01,1.086306e+00,1.756172e+00,1.822792e+00,...,7.688348e-01,1.140853e+00,2.673188e+00,9.516164e-01,8.500538e-01,1.513627e-01,2.773990e+00,2.800536e-01,1.999257e-01,5.718548e-01
min,-1.000000e+00,-1.000000e+00,9.586066e-01,1.017195e+00,1.017208e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,...,5.883444e-06,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.000000e+00,5.883444e-06,6.534956e-01,7.821732e-01
25%,-1.000000e+00,-1.000000e+00,1.089709e+00,2.598649e+00,4.201412e+00,1.336561e+00,2.215969e+00,3.151457e+00,1.401545e+00,1.765791e+00,...,1.254214e+00,6.000000e+00,8.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,1.700000e+01,5.825748e-01,2.158632e+00,3.200000e+00
50%,-1.000000e+00,-1.000000e+00,1.094119e+00,2.853414e+00,4.734151e+00,1.501337e+00,2.528806e+00,3.803336e+00,2.388518e+00,2.628668e+00,...,1.733884e+00,7.000000e+00,1.000000e+01,1.000000e+00,1.000000e+00,0.000000e+00,1.900000e+01,7.652321e-01,2.266137e+00,3.520592e+00
75%,-1.000000e+00,-1.000000e+00,1.100299e+00,3.218238e+00,5.574344e+00,1.529120e+00,2.927891e+00,4.601007e+00,3.151907e+00,3.484171e+00,...,2.302506e+00,7.000000e+00,1.200000e+01,1.000000e+00,2.000000e+00,0.000000e+00,2.100000e+01,9.629849e-01,2.391795e+00,3.955266e+00
max,7.805846e+00,8.341639e+00,1.568057e+00,6.885714e+00,1.204039e+01,3.976531e+00,6.523529e+00,1.115916e+01,1.047548e+01,1.047548e+01,...,6.389156e+00,9.000000e+00,2.000000e+01,7.000000e+00,5.000000e+00,6.000000e+00,2.900000e+01,1.897077e+00,3.941514e+00,7.130415e+00


In [20]:
from collections import namedtuple

SpinProp = namedtuple('SpinProp',['mu','spin','NMR'] )
"""
https://en.wikipedia.org/wiki/Nuclear_magnetic_moment

mu: magnetic dipole moment
spin: nuclear spin number
NMR: NMR senstiivity relative to H
"""
AtomicSpinProp = {
    'H': SpinProp(2.79284734, 0.5, 1),
    'C': SpinProp(0.7024118, 0.5, 0.016),
    'O': SpinProp(-1.89379,2.5,0.037),
    'N': SpinProp(0.40376100, 1, 0.001),
    'F': SpinProp(2.628868, 0.5, 0.83)
}

def AddSpinProp(df):
    df = df.assign(mu= lambda x: x.atom_end_type.map(lambda element: AtomicSpinProp[element].mu),
             spin= lambda x: x.atom_end_type.map(lambda element: AtomicSpinProp[element].spin),
             NMR=lambda x: x.atom_end_type.map(lambda element: AtomicSpinProp[element].NMR))
    return df

In [21]:
#train = AddSpinProp(train)
#test=AddSpinProp(test)
#PrintDataframe(train.head())

In [22]:
#MolProps.to_csv('MolecularProperties.csv')
#structures.to_csv('structures_extended.csv', index=False)
train_extend.to_csv('train_extend.csv', index=False)
#test_extend.to_csv('test_extend.csv', index=False)

/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [23]:
len(train_extend.bond_2.unique())*len(train_extend.bond_3.unique())

16

In [24]:
len(train_extend.atom_0_type2.unique())*len(train_extend.atom_end_type2.unique())*len(train_extend.atom_2_type.unique())

504

In [25]:
len(train_extend.atom_0_type2.unique())*len(train_extend.atom_end_type2.unique())*len(train_extend.atom_2_type.unique())*len(train_extend.atom_3_type.unique())

8064

In [26]:
len(train_extend)

4658147